In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

from IPython.display import clear_output
from datetime import datetime

In [2]:
param_date = '20240724'

In [3]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel(f'./{param_date}/3.type.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['Group No'] = input_['Group No'].astype(int)
input_['Subgroup No'] = input_['Subgroup No'].astype(int)
input_['Type No'] = input_['Type No'].astype(int)

print(f'总数量：{len(input_)}')
print()

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)

# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'

        # = = = = = = = = = = = = = = =

        import requests
        import json

        import sys
        sys.path.append('../../00.Tools')
        from crawler_configuration import get_header, get_proxy

        payload = {'jsn': {'tab': 'extras',
                           'groupname': input_.loc[a, 'Group'],
                           'subgroupname': input_.loc[a, 'Subgroup'],
                           'parttype': input_.loc[a, 'Type Code'],
                           'nodetype': 'parttype'}}

        data = {'func': 'navnode_fetch',
                'payload': json.dumps(payload),
                'api_json_request': '1'}

        try:
            b = 0
            while True:

                while True:
                    b += 1
                    try:
                        resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                                             data=data,
                                             headers=get_header(),
                                             proxies=get_proxy(),
                                             timeout=(10, 10))
                        break
                    except:
                        continue

                if resp.status_code == 200:
                    json_ = resp.json()
                    if 'redirect_to_url' in json_:
                        continue
                    elif json_['jsn']['tab'] == 'extras' and json_['jsn']['groupname'] == input_.loc[a, 'Group'] and json_['jsn']['subgroupname'] == input_.loc[a, 'Subgroup'] and json_['jsn']['parttype'] == input_.loc[a, 'Type Code'] and json_['jsn']['nodetype'] == 'parttype' and json_['jsn']['parttypedesc'] == input_.loc[a, 'Type'] and 'collected_javascript' in json_:
                        break

            # = = = = = = = = = = = = = = =

            from bs4 import BeautifulSoup
            from lxml import etree

            soup = BeautifulSoup(json_['html_fill_sections']['navchildren[]'], 'lxml')
            html = etree.HTML(str(soup))

            # = = = = = = = = = = = = = = =

            list_tbody = html.xpath('//tbody[contains(@class, "listing-inner") and (contains(@class, "altrow-a-1") or contains(@class, "altrow-a-0"))]')

            # = = = = = = = = = = = = = = =

            if not list_tbody:
                raise

            # = = = = = = = = = = = = = = =

            list_list_note = [tbody.xpath('./tr[1]/td/div[@class="listing-text-row-moreinfo-truck"]/span[@class="span-link-underline-remover"]/text()') for tbody in list_tbody]
            list_note = ['\n'.join([note.strip() for note in list_note]) for list_note in list_list_note]

            # = = = = = = = = = = = = = = =

            list_list_info = [tbody.xpath('./tr[1]/td/div[@class="listing-text-row-moreinfo-truck"]/a[@class="ra-btn ra-btn-moreinfo"]/@href') for tbody in list_tbody]
            list_info = [list_info[0].strip() if list_info and list_info[0].startswith('https://www.rockauto.com/en/moreinfo.php?') else '' for list_info in list_list_info]

            # = = = = = = = = = = = = = = =

            import re

            list_note_2 = [etree.tostring(tbody.xpath('./tr[1]/td/div[@class="listing-text-row"]')[0]).decode('utf-8') for tbody in list_tbody]
            list_note_2 = [re.sub(r'<.*?>', '', note_2.strip()) for note_2 in list_note_2]
            list_note_2 = [re.sub(r' +', ' ', note_2.strip()) for note_2 in list_note_2]

            # = = = = = = = = = = = = = = =

            list_json = [json.loads(tbody.xpath('./tr[1]/td[contains(@class, "listing-inner-content")]/input[contains(@id, "listing_data_essential[__GIP__")]/@value')[0]) for tbody in list_tbody]

            # = = = = = = = = = = = = = = =

            list_key = [json_['partkey'] for json_ in list_json]

            # = = = = = = = = = = = = = = =

            list_json = [json.loads(tbody.xpath('./tr[1]/td/input[contains(@id, "listing_data_supplemental[")]/@value')[0]) for tbody in list_tbody]

            # = = = = = = = = = = = = = = =

            list_part_number = [json_['partnumber'] for json_ in list_json]

            # = = = = = = = = = = = = = = =

            list_manufacturer = [json_['catalogname'] for json_ in list_json]

            # = = = = = = = = = = = = = = =

            list_partition = [json_['sortgrouptext'] for json_ in list_json]

            # = = = = = = = = = = = = = = =

            list_parameter = [json_['paramdesc'] for json_ in list_json]

            # = = = = = = = = = = = = = = =

            list_list_oem = [tbody.xpath('./tr[1]/td/span[contains(@title, "Replaces these Alternate")]/text()') for tbody in list_tbody]
            list_oem = [','.join(list_oem) for list_oem in list_list_oem]
            list_oem = [';'.join([oem_split.strip() for oem_split in oem.split(',')]) for oem in list_oem]

            # = = = = = = = = = = = = = = =

            list_list_src = [json.loads(tbody.xpath('./tr[1]/td/div/input[contains(@id, "jsninlineimg[")]/@value')[0])['Slots'] if tbody.xpath('./tr[1]/td/div/input[contains(@id, "jsninlineimg[")]/@value') else [] for tbody in list_tbody]

            # = = = = = = = = = = = = = = =

            list_dict_src = []
            for list_src in list_list_src:
                dict_src = {}
                for i, src in zip(range(len(list_src)), list_src):
                    dict_src[str(i)] = 'https://www.rockauto.com' + src['ImageData']['Full'].strip()

                list_dict_src.append(dict_src)

            # = = = = = = = = = = = = = = =

            df_temp = pd.DataFrame({'Group No': input_.loc[a, 'Group No'],
                                    'Group': input_.loc[a, 'Group'],
                                    'Subgroup No': input_.loc[a, 'Subgroup No'],
                                    'Subgroup': input_.loc[a, 'Subgroup'],
                                    'Type No': input_.loc[a, 'Type No'],
                                    'Type': input_.loc[a, 'Type'],
                                    'Type Code': input_.loc[a, 'Type Code'],
                                    'Tab 3 Url': input_.loc[a, 'Tab 3 Url'],
                                    'Part No': [i+1 for i in range(len(list_part_number))],
                                    'Partition': list_partition,
                                    'Manufacturer': list_manufacturer,
                                    'Part Number': list_part_number,
                                    'Key': list_key,
                                    'JOIN_MPNTCK': [f'''{manufacturer};{part_number};{input_.loc[a, 'Type Code']};{key}''' for manufacturer, part_number, key in zip(list_manufacturer, list_part_number, list_key)],
                                    'Tab 3 Note 1': list_note,
                                    'Tab 3 Note 2': list_note_2,
                                    'Parameter': list_parameter,
                                    'OEM': list_oem,
                                    'Info': list_info,
                                    'Json_Src': [json.dumps(dict_src) for dict_src in list_dict_src]})

            output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            crawler_status = 'ok'
        except:
            df_temp = pd.DataFrame([{'Group No': input_.loc[a, 'Group No'],
                                     'Group': input_.loc[a, 'Group'],
                                     'Subgroup No': input_.loc[a, 'Subgroup No'],
                                     'Subgroup': input_.loc[a, 'Subgroup'],
                                     'Type No': input_.loc[a, 'Type No'],
                                     'Type': input_.loc[a, 'Type'],
                                     'Type Code': input_.loc[a, 'Type Code'],
                                     'Tab 3 Url': input_.loc[a, 'Tab 3 Url']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

        # = = = = = = = = = = = = = = =

        if work.qsize() % 1_000 == 0 and work.qsize() != 0:
            clear_output()

        print(f'''[状态：{crawler_status}，尝试次数：{b}] - {input_.loc[a, 'Group No']}.{input_.loc[a, 'Group']} > {input_.loc[a, 'Subgroup No']}.{input_.loc[a, 'Subgroup']} > {input_.loc[a, 'Type No']}.{input_.loc[a, 'Type']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(12):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['Group No', 'Subgroup No', 'Type No', 'Part No'],
                                                ascending=[True, True, True, True],
                                                ignore_index=True)
    output_correct.to_excel(f'''./{param_date}/4.part_1-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
if not output_error.empty:
    output_error = output_error.sort_values(by=['Group No', 'Subgroup No', 'Type No'],
                                            ascending=[True, True, True],
                                            ignore_index=True)
    output_error.to_excel(f'./{param_date}/part_1_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

[状态：ok，尝试次数：84] - 10.Electrical-Switch & Relay > 2.Relay > 2.Multi-Function Relay
[剩余数量：1000] - [当前时间：17:56:29]

[状态：ok，尝试次数：86] - 10.Electrical-Switch & Relay > 3.Specialty Switch > 6.Hazard Warning Switch
[剩余数量：999] - [当前时间：17:56:29]

[状态：ok，尝试次数：27] - 10.Electrical-Switch & Relay > 4.Switch > 5.Toggle Switch
[剩余数量：998] - [当前时间：17:56:30]

[状态：ok，尝试次数：24] - 11.Engine > 1.Cylinder Head > 1.Cylinder Head Bolt Washer
[剩余数量：997] - [当前时间：17:56:30]

[状态：ok，尝试次数：38] - 10.Electrical-Switch & Relay > 5.Tool > 1.Oil Pressure Switch Socket
[剩余数量：996] - [当前时间：17:56:30]

[状态：ok，尝试次数：15] - 11.Engine > 2.Dipstick > 1.Oil Dipstick / Tube
[剩余数量：995] - [当前时间：17:56:30]

[状态：ok，尝试次数：2] - 11.Engine > 6.Heater > 1.Engine Block Heater
[剩余数量：994] - [当前时间：17:56:31]

[状态：ok，尝试次数：23] - 11.Engine > 3.Fluid / Chemical > 1.Camshaft Assembly Paste
[剩余数量：993] - [当前时间：17:56:32]

[状态：ok，尝试次数：12] - 11.Engine > 3.Fluid / Chemical > 3.Degreaser
[剩余数量：992] - [当前时间：17:56:32]

[状态：ok，尝试次数：45] - 11.Engine > 3.Fluid / Chemica